# 第二章——文字向量化

In [1]:
# 如果没有transformers则安装
# ! pip install transformers

In [2]:
from transformers import GPT2Tokenizer, GPT2Model

# 初始化分词器和模型
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

# 待向量化的文本
text = "LLM with me"
# 分词并转换为索引
inputs = tokenizer(text, return_tensors="pt")
# 输出token及其对应的索引
print(inputs)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

{'input_ids': tensor([[3069,   44,  351,  502]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [3]:
text = "LLM"
inputs = tokenizer(text, return_tensors="pt")
print(inputs)

# 查看索引对应的token
print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

{'input_ids': tensor([[3069,   44]]), 'attention_mask': tensor([[1, 1]])}
['LL', 'M']


In [4]:
from transformers import GPT2Tokenizer

# 初始化分词器
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# 获取词汇表的大小
vocab_size = len(tokenizer)
print(f"The vocabulary size of GPT2Tokenizer is: {vocab_size}")

The vocabulary size of GPT2Tokenizer is: 50257


In [5]:
from transformers import GPT2Tokenizer, GPT2Model

# 初始化分词器和模型
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
# 待处理的文本
text = "LLM with me"
# 分词并转换为索引
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
# 获取模型的嵌入层
embeddings = model.get_input_embeddings()
# 将索引转换为嵌入向量
input_embeddings = embeddings(input_ids)
print(input_embeddings)
print(input_embeddings.shape)

tensor([[[ 0.2509, -0.1875,  0.1510,  ...,  0.1094,  0.1639,  0.3363],
         [-0.0159, -0.1385,  0.2203,  ..., -0.0501,  0.0990, -0.0755],
         [ 0.0644,  0.0104,  0.0293,  ...,  0.0400,  0.1087,  0.0350],
         [ 0.1515, -0.0247,  0.0936,  ..., -0.1684,  0.1065,  0.0572]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 4, 768])


In [6]:
from transformers import GPT2Tokenizer, GPT2Config, GPT2Model
import torch

# 初始化分词器& 创建模型配置 & 初始化模型
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
config = GPT2Config(vocab_size=len(tokenizer), n_embd=768, n_layer=12, n_head=12)
model = GPT2Model(config)
# 待处理的文本 & 分词并转换为索引
text = "LLM with me"
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
print(input_ids)
# 获取模型的嵌入层
embeddings = model.get_input_embeddings()
# 将索引转换为嵌入向量
input_embeddings = embeddings(input_ids)

print(input_embeddings)
print(input_embeddings.shape)

tensor([[3069,   44,  351,  502]])
tensor([[[-0.0286,  0.0050, -0.0065,  ..., -0.0037,  0.0252, -0.0502],
         [-0.0149, -0.0292,  0.0061,  ...,  0.0174,  0.0070,  0.0024],
         [ 0.0027,  0.0040,  0.0024,  ..., -0.0140, -0.0124,  0.0112],
         [-0.0026, -0.0169, -0.0193,  ...,  0.0246, -0.0027,  0.0186]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 4, 768])


In [7]:
# from tokenizers import Tokenizer
# from tokenizers.models import BPE
# from tokenizers.trainers import BpeTrainer
# from tokenizers.pre_tokenizers import Whitespace

# # 创建一个空的BPE分词器 & 使用空格进行预分词 & 创建一个分词器训练器
# tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
# tokenizer.pre_tokenizer = Whitespace()
# trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

# # 准备一些训练数据，这里我们只用一句话 & 在实际应用中，你需要大量的文本数据
# train_data = ["LLM with me"]

# # 训练分词器
# tokenizer.train_from_iterator(train_data, trainer)
# # 保存分词器到文件
# tokenizer.save("custom_tokenizer.json")
# # 测试分词器
# output = tokenizer.encode("LLM with me")
# print(output.tokens)

In [8]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# 创建一个空的BPE分词器 & 使用空格进行预分词 & 创建一个分词器训练器
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]", "<EOS>"])  # 添加两个占位来解决不认识的词和结束语

# 准备一些训练数据，这里我们只用一句话 & 在实际应用中，你需要大量的文本数据
train_data = ["LLM with me"]
# 训练分词器
tokenizer.train_from_iterator(train_data, trainer)
# 保存分词器到文件
tokenizer.save("custom_tokenizer.json")
# 测试分词器
output = tokenizer.encode("LLM with me <EOS>")
print(output.tokens)


['LLM', 'with', 'me', '<EOS>']




In [9]:
from transformers import PreTrainedTokenizerFast

# 获取文本的索引
input_ids = output.ids
print(input_ids)

# 加载自定义分词器 & 编码文本并返回PyTorch张量
tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
print(input_ids)

[14, 15, 12, 1]
tensor([[14, 15, 12]])


In [10]:
# 查看分词器的词汇表
vocab = tokenizer.get_vocab()
print(vocab)

{'e': 4, 'it': 11, 'm': 7, 'h': 5, 'w': 9, 'LLM': 14, '[UNK]': 0, 'i': 6, 'LL': 10, 'wit': 13, 'M': 3, 'with': 15, 'L': 2, 't': 8, '<EOS>': 1, 'me': 12}


In [11]:
from transformers import PreTrainedTokenizerFast
from transformers import GPT2Config, GPT2Model

# 加载自定义分词器
tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")

# 创建模型配置 & 初始化模型
config = GPT2Config(vocab_size=tokenizer.vocab_size, n_embd=768, n_layer=12, n_head=12)
model = GPT2Model(config)

# 待处理的文本 & 分词并转换为索引
text = "LLM with me"
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]

# 获取模型的嵌入层 & 将索引转换为嵌入向量
embeddings = model.get_input_embeddings()
input_embeddings = embeddings(input_ids)
print(input_embeddings)
print(input_embeddings.shape)

tensor([[[-0.0054,  0.0349, -0.0085,  ..., -0.0360, -0.0266, -0.0049],
         [-0.0047, -0.0010,  0.0164,  ..., -0.0157, -0.0245, -0.0222],
         [-0.0183,  0.0165, -0.0246,  ..., -0.0089,  0.0305, -0.0066]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 3, 768])


In [12]:
import torch
from transformers import PreTrainedTokenizerFast

# 加载自定义分词器
tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")
# 待处理的文本 & 分词并转换为索引
text = "LLM with me"
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]

# 假设我们的自定义分词器有一个很小的词汇表
vocab_size = tokenizer.vocab_size  # 从自定义分词器获取词汇表大小
n_embd = 10  # 设置嵌入维度为10
# 创建一个随机初始化的嵌入矩阵，这里我们使用正态分布随机初始化，与实际模型初始化类似
embedding_matrix = torch.randn(vocab_size, n_embd)
token_indices = input_ids[0]  # 假设input_ids是一个包含索引的张量
token_embeddings = embedding_matrix[token_indices]
print(token_embeddings)
print(token_embeddings.shape)

tensor([[-0.0734,  0.9592,  1.3814,  2.3693,  2.3262, -1.0959,  0.7059, -0.0506,
         -0.0729, -1.1791],
        [-0.5122,  0.6106, -0.3071,  0.4347,  0.2445,  2.0369,  0.3645, -0.4135,
         -0.5863,  1.2864],
        [-2.0330,  0.1906, -0.1990, -0.4726,  2.1953,  1.0321, -2.0593, -0.5847,
         -0.3605, -1.9308]])
torch.Size([3, 10])


In [13]:
print(embedding_matrix[13])
print(embedding_matrix[14])
print(embedding_matrix[11])

tensor([ 0.8079,  0.3908, -1.4927,  0.7122,  1.9811, -0.1907, -0.7898, -0.7037,
         2.5170,  1.1116])
tensor([-0.0734,  0.9592,  1.3814,  2.3693,  2.3262, -1.0959,  0.7059, -0.0506,
        -0.0729, -1.1791])
tensor([-1.0125, -0.0436, -0.1076, -0.2520,  1.0112, -1.5468, -0.2592, -1.1768,
         1.1942, -0.6323])


In [14]:
import torch
from transformers import PreTrainedTokenizerFast

# 加载自定义分词器
tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")
# 待处理的文本 & 分词并转换为索引
text = "LLM with me"
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
vocab_size = tokenizer.vocab_size; n_embd = 10
# 创建一个随机初始化的嵌入矩阵，这里我们使用Xavier初始化
embedding_matrix = torch.empty(vocab_size, n_embd)
torch.nn.init.xavier_uniform_(embedding_matrix)  # 使用Xavier均匀初始化

token_indices = input_ids[0]  # 假设input_ids是一个包含索引的张量
token_embeddings = embedding_matrix[token_indices]
print(token_embeddings)
print(token_embeddings.shape)

tensor([[-0.0731, -0.0870, -0.0354, -0.1912,  0.0068,  0.1697, -0.3924,  0.0451,
         -0.1344,  0.1391],
        [-0.1168,  0.2037, -0.2497,  0.3482, -0.0643, -0.1626,  0.0857, -0.3652,
          0.1890, -0.2604],
        [ 0.1953, -0.1465, -0.0192,  0.3124,  0.4406, -0.4748, -0.3842, -0.3123,
         -0.4185, -0.1526]])
torch.Size([3, 10])


In [15]:
print(embedding_matrix[13])
print(embedding_matrix[14])
print(embedding_matrix[11])

tensor([-0.3125,  0.1946, -0.0692, -0.3138, -0.1735,  0.0426, -0.3563, -0.1339,
         0.2658, -0.1542])
tensor([-0.0731, -0.0870, -0.0354, -0.1912,  0.0068,  0.1697, -0.3924,  0.0451,
        -0.1344,  0.1391])
tensor([ 0.2595, -0.1769,  0.2184, -0.1655,  0.1156,  0.3385, -0.1067, -0.3338,
        -0.3268, -0.2799])


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

vocab_size = tokenizer.vocab_size; n_embd = 10
embedding_matrix = torch.empty(vocab_size, n_embd)
nn.init.xavier_uniform_(embedding_matrix)
# 定义一个简化版的GPT模型
class SimpleGPT(nn.Module):
    def __init__(self, vocab_size, n_embd):
        super(SimpleGPT, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, n_embd)
        self.ffn = nn.Linear(n_embd, n_embd)
        self.logits = nn.Linear(n_embd, vocab_size)
        nn.init.xavier_uniform_(self.embeddings.weight)  # 使用Xavier初始化嵌入层
    
    def forward(self, input_ids):
        x = self.embeddings(input_ids)  # 嵌入层
        x = self.ffn(x)  # 前馈网络
        logits = self.logits(x)  # 输出层
        return logits

# 创建模型实例 & 定义损失函数和优化器
model = SimpleGPT(vocab_size, n_embd)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 假设我们有一些训练数据
input_ids = torch.tensor([[1, 2, 3, 4], [2, 3, 4, 5]])  # 示例输入
labels = torch.tensor([[2, 3, 4, 5], [3, 4, 5, 6]])  # 示例目标

# 训练循环
for epoch in range(100):  # 假设训练100个epoch
    logits = model(input_ids)  # 前向传播
    loss = loss_fn(logits.view(-1, vocab_size), labels.view(-1))  # 计算损失
    # 反向传播
    optimizer.zero_grad()  
    loss.backward()
    # 梯度下降
    optimizer.step()
    # 打印损失
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

Epoch 10, Loss: 2.7324118614196777
Epoch 20, Loss: 2.657238245010376
Epoch 30, Loss: 2.580580472946167
Epoch 40, Loss: 2.500619888305664
Epoch 50, Loss: 2.415583848953247
Epoch 60, Loss: 2.3237650394439697
Epoch 70, Loss: 2.2237038612365723
Epoch 80, Loss: 2.1143651008605957
Epoch 90, Loss: 1.9953210353851318
Epoch 100, Loss: 1.8669122457504272


In [17]:
token_indices = input_ids[0]
token_embeddings = model.embeddings(token_indices)
print(token_embeddings)
print(token_embeddings.shape)

tensor([[ 0.3168,  0.1598, -0.2357, -0.1286,  0.4422,  0.0902, -0.2156,  0.1508,
         -0.3751, -0.4240],
        [ 0.3838, -0.2698,  0.2582, -0.1764,  0.4416, -0.0557,  0.5702,  0.3589,
         -0.0439,  0.4755],
        [ 0.0883, -0.5616, -0.4737, -0.1625,  0.4614, -0.1707, -0.3864, -0.3232,
         -0.1757,  0.2665],
        [-0.4491,  0.5912,  0.0080,  0.0760,  0.0837, -0.4634, -0.5850, -0.4476,
         -0.4615, -0.2961]], grad_fn=<EmbeddingBackward0>)
torch.Size([4, 10])


In [18]:
import torch.nn.functional as F
# 假设model是训练好的模型实例 & 假设tokenizer是加载好的分词器
model.eval() # 将模型设置为评估模式
input_text = "LLM with me"  # 输入文本
input_ids = tokenizer.encode(input_text, return_tensors="pt")  # 将文本编码为token索引
temperature = 0.7  # 设置温度参数 & 一般设置为0到1之间的值
generated_text = input_text + " A:"
for _ in range(50):  # 假设我们想生成50个单词
    with torch.no_grad():  # 不需要计算梯度
        logits = model(input_ids)
        logits = logits / temperature  # 应用温度调整
        # 使用softmax函数将logits转换为概率分布 & 根据概率分布随机选择下一个单词
        probabilities = F.softmax(logits[:, -1, :], dim=-1)
        predicted_id = torch.multinomial(probabilities, num_samples=1)
        # 将预测的token添加到输入序列中 & 将预测的token解码为文本并添加到生成的文本中
        input_ids = torch.cat((input_ids, predicted_id), dim=1)
        generated_text += tokenizer.decode(predicted_id[0])

print(generated_text)
eos_token = '<EOS>'  # 在生成文本后根据<EOS>进行切割
generated_text_parts = generated_text.split(eos_token)
final_text = generated_text_parts[0] + eos_token if len(generated_text_parts) > 1 else generated_text_parts[0]
print(final_text)

LLM with me A:[UNK]iLLwitMimeLMethihitMehw<EOS>LLMwitMmmMwitLLLLLLMLMLMiwitLLmLwithe[UNK]LLhi[UNK]witLLM
LLM with me A:[UNK]iLLwitMimeLMethihitMehw<EOS>
